<a href="https://colab.research.google.com/github/Elbereth-Elentari/Book_recommender/blob/master/Scrape_BUW_catalogue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install scrapy
!pip3 install bs4

     |████████████████████████████████| 245kB 3.4MB/s 
     |████████████████████████████████| 235kB 9.2MB/s 
     |████████████████████████████████| 3.1MB 48kB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 3.2MB 37.3MB/s 
     |████████████████████████████████| 2.7MB 46.1MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11515 sha256=8cecf2f7c5ec214f4d5804c8045fdfe80468bbde52993ba42deaab34c29a3503
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
  Created wheel for protego: filename=Protego-0.1.16-cp36-none-any.whl size=7765 sha256=bb65de8027da8fbad30d94cd2580fde1cfac7e659336fee4a7a7dffadf0fcd73
  Stored in directory: /root/.cache/pip/wheels/51/01/d1/4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
Successfully built PyDispatcher protego


In [ ]:
import scrapy
from bs4 import BeautifulSoup
import json
import re
import logging


class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('/content/drive/My Drive/Library_catalogue.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item


class BookSpider(scrapy.Spider):
    name = 'books'
    start_urls = ['https://chamo.buw.uw.edu.pl/search/query?match_1=MUST&field_1=text&match_2=PHRASE&field_2=text&match_3=SHOULD&field_3=text&match_4=NOT&field_4=text&filter_loc=10000&filter_loc=10002&filter_format=book&filter_lang=pol&filter_lang=eng&theme=system']
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 100},
        'FEED_FORMAT':'json',
        'FEED_URI': '/content/drive/My Drive/Library_catalogue.json'         
    }


    def start_requests(self):
        urls = [f'https://chamo.buw.uw.edu.pl/search/query?filter_format=book&filter_lang=pol&filter_lang=eng&filter_loc=10000&filter_loc=10002&sort=dateCreated&pageNumber={page}&theme=system' for page in range(74856)]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)


    def parse(self, response):
        soup = BeautifulSoup(response.body, 'html.parser')
        records = soup.find(class_='records').find_all('div', class_='record')
        
        author, storage, WD_signature, publisher,  = '', '', '', ''
        pages, year = 0, 0
        for record in records:
            if 'Adres wyd.' in record.text and 'Opis fiz.' in record.text:
                title = record.find(class_='title').text
                title = re.sub(r' [=/].+', '', title)
                try: author = record.find(class_='author').text
                except: pass
                if 'BUW Magazyn' in record.text: storage = 'magazyn'

                infos = record.find_all('tr')
                for info in infos:
                    if 'Klasyfikacja WD' in info.text:
                        WD_signature = info.a.text
                    elif 'Adres wyd.' in info.text:
                        publisher_candidates = info.find_all('span')
                        for publisher in publisher_candidates:
                            if ('class' in publisher and publisher['class'] != 'highlight') or 'class' not in publisher:
                                publisher_with_colon = re.search(r".+ : ?([\w .']+),?.*(\d{4})", publisher.text)
                                if publisher_with_colon:
                                    publisher = publisher_with_colon.group(1)
                                    year = int(publisher_with_colon.group(2))
                                else:
                                    publisher = re.search(r".+? ([\w .']+),?.*(\d{4})", publisher.text)
                                    if publisher:
                                        year = int(publisher.group(2))
                                        publisher = publisher.group(1)
                    elif 'Opis fiz.' in info.text:
                        pages = info.find('span').text
                        pages = re.sub(r'\[.+?\]', '', pages)
                        pages = int(re.search(r'\d+', pages).group(0))

                yield {'title': title,
                       'author': author,
                       'storage': storage,
                       'WD_signature': WD_signature,
                       'publisher': publisher,
                       'year': year,
                       'pages': pages}

In [ ]:
!scrapy startproject buw
!cd buw

In [ ]:
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

process = CrawlerProcess(get_project_settings())

process.crawl(BookSpider)
process.start()